In [1]:
pip install torch torchvision transformers flask scikit-learn pandas numpy pillow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import os

# Khởi tạo mô hình ViT và trình tiền xử lý
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Hàm để tải và tiền xử lý ảnh
def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=img, return_tensors="pt")
    return inputs['pixel_values']


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [3]:
# Đọc dữ liệu từ CSV
df = pd.read_csv('./newImageProduct.csv')  
features = {}

# Trích xuất đặc trưng cho từng ảnh
for idx, row in df.iterrows():
    image_path = row['imagepath']
    product_id = row['productid']
    
    # Trích xuất đặc trưng
    pixel_values = load_and_preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(pixel_values)
        feature_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    
    # Bình thường hóa đặc trưng và lưu trữ
    features[product_id] = normalize([feature_vector])[0]

# Lưu đặc trưng vào file .npz
np.savez('vit_features.npz', **features)


In [3]:
import torch
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image, ImageOps
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import os

# Initialize ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Image preprocessing function
def load_and_preprocess_image(image_path):
    # Load image and apply preprocessing
    img = Image.open(image_path).convert("RGB")
    
    # Resize and apply any additional preprocessing if needed
    img = ImageOps.exif_transpose(img)  # Correct orientation
    img = img.resize((224, 224))  # Resizing for ViT input
    
    # Augmentation (optional): Horizontal Flip, Rotate
    img = img.transpose(Image.FLIP_LEFT_RIGHT) if np.random.rand() > 0.5 else img
    img = img.rotate(np.random.uniform(-10, 10))  # Small random rotation

    # Use feature extractor to process the image for ViT
    inputs = feature_extractor(images=img, return_tensors="pt")
    return inputs['pixel_values']

# Load data from CSV
df = pd.read_csv('./src/newImageProduct.csv')
features = {}

# Extract features for each image
for idx, row in df.iterrows():
    image_path = row['imagepath']
    product_id = row['productid']
    
    try:
        # Extract features
        pixel_values = load_and_preprocess_image(image_path)
        with torch.no_grad():
            outputs = model(pixel_values)
            feature_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        
        # Normalize and store the features
        features[product_id] = normalize([feature_vector])[0]
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

# Save features to .npz file
np.savez('vit_features.npz', **features)
print("Feature extraction completed and saved.")


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Feature extraction completed and saved.


In [4]:
import torch
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import os

# Khởi tạo ViT model và feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Hàm tiền xử lý và tăng cường ảnh
def load_and_preprocess_image(image_path):
    # Tải ảnh và chuyển về RGB
    img = Image.open(image_path).convert("RGB")
    
    # Điều chỉnh orientation và resize ảnh
    img = ImageOps.exif_transpose(img)
    img = img.resize((224, 224))
    
    # Tăng cường hình ảnh: Lật ngang, Xoay, Điều chỉnh độ sáng và tương phản, Làm mờ Gaussian
    if np.random.rand() > 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    img = img.rotate(np.random.uniform(-10, 10))  # Xoay ngẫu nhiên trong khoảng -10 đến 10 độ

    # Điều chỉnh độ sáng và độ tương phản
    img = ImageEnhance.Brightness(img).enhance(np.random.uniform(0.8, 1.2))
    img = ImageEnhance.Contrast(img).enhance(np.random.uniform(0.8, 1.2))

    # Áp dụng Gaussian Blur nhẹ để giảm nhiễu
    img = img.filter(ImageFilter.GaussianBlur(radius=0.5))

    # Sử dụng feature extractor để chuẩn bị dữ liệu cho ViT
    inputs = feature_extractor(images=img, return_tensors="pt")
    return inputs['pixel_values']

# Tải dữ liệu từ CSV
df = pd.read_csv('./src/newImageProduct.csv')
features = {}

# Trích xuất đặc trưng cho từng ảnh
for idx, row in df.iterrows():
    image_path = row['imagepath']
    product_id = row['productid']
    
    try:
        # Trích xuất đặc trưng
        pixel_values = load_and_preprocess_image(image_path)
        with torch.no_grad():
            outputs = model(pixel_values)
            feature_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        
        # Chuẩn hóa và lưu trữ đặc trưng
        features[product_id] = normalize([feature_vector])[0]
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {image_path}: {e}")

# Lưu đặc trưng vào file .npz
np.savez('vit_features_preprocess_image.npz', **features)
print("Hoàn tất trích xuất đặc trưng và lưu vào file.")


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Hoàn tất trích xuất đặc trưng và lưu vào file.


In [6]:
import torch
from transformers import ViTImageProcessor, ViTModel
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
import random
from torchvision.transforms import ColorJitter, RandomResizedCrop

# Initialize ViT model and image processor
image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Load data from CSV
df = pd.read_csv('./src/newImageProduct.csv')
features = {}

# Extract features for each image
for idx, row in df.iterrows():
    image_path = row['imagepath']
    product_id = row['productid']
    
    try:
        # Load and preprocess the image
        img = Image.open(image_path).convert("RGB")
        img = ImageOps.exif_transpose(img)  # Correct orientation
        img = img.resize((224, 224))  # Resize for ViT input

        # Augmentations
        if random.random() > 0.5:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)  # Horizontal flip
        img = img.rotate(random.uniform(-10, 10))  # Small random rotation
        img = ImageEnhance.Brightness(img).enhance(random.uniform(0.8, 1.2))  # Adjust brightness
        img = ImageEnhance.Contrast(img).enhance(random.uniform(0.8, 1.2))  # Adjust contrast
        img = img.filter(ImageFilter.GaussianBlur(radius=0.5))  # Gaussian blur

        # Additional augmentations from torchvision
        img = ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05)(img)
        img = RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.33))(img)

        inputs = image_processor(images=img, return_tensors="pt")
        
        # Extract features from the image
        with torch.no_grad():
            outputs = model(**inputs)
            feature_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

        # Store the feature vector
        features[product_id] = feature_vector
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

# Prepare for PCA: ensure we have more than one feature vector
if len(features) > 1:
    # Convert features to a numpy array for PCA
    feature_matrix = np.array(list(features.values()))
    
    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=128)
    reduced_features = pca.fit_transform(feature_matrix)

    # Normalize the reduced features
    normalized_features = normalize(reduced_features)
    
    # Create a dictionary with normalized features
    normalized_features_dict = {product_id: normalized_features[i] for i, product_id in enumerate(features.keys())}
    
    # Save features to .npz file
    np.savez('vit_features_with_pca.npz', **normalized_features_dict)
    print("Feature extraction with PCA completed and saved.")
else:
    print("Not enough features for PCA. At least 2 feature vectors are required.")


Feature extraction with PCA completed and saved.
